# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [87]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [90]:
citiesDf = pd.read_csv("Output/cities.csv")
citiesDf

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.1500,-70.9167,4.06,75,0,4.63,CL,1624066728
1,1,Hasaki,35.7333,140.8333,21.27,84,100,4.81,JP,1624066831
2,2,Shirokiy,49.7580,129.5230,19.40,70,95,8.49,RU,1624066846
3,3,Puerto Escondido,15.8500,-97.0667,26.46,57,90,0.00,MX,1624066846
4,4,Nikolskoye,59.7035,30.7861,17.94,80,0,2.29,RU,1624066805
...,...,...,...,...,...,...,...,...,...,...
558,558,Lingao,19.9103,109.6859,32.70,56,12,6.42,CN,1624066981
559,559,Sonthofen,47.5167,10.2833,20.21,87,87,0.45,DE,1624066981
560,560,Dwārāhāt,29.7833,79.4333,17.71,98,100,0.99,IN,1624066981
561,561,Kenai,60.5544,-151.2583,18.05,52,90,5.14,US,1624066747


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [91]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [92]:
locations=[]

for index,row in citiesDf.iterrows():
    locations.insert(index, (row['Latitude'], row['Longitude']))
humidity= citiesDf['Humidity']
print(humidity)

0      75
1      84
2      70
3      57
4      80
       ..
558    56
559    87
560    98
561    52
562    95
Name: Humidity, Length: 563, dtype: int64


In [93]:


figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)
#print(fig)

fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [116]:
hotel_df = citiesDf.loc[(citiesDf['Max Temperature'] > 24) & (citiesDf['Max Temperature'] < 26) & (citiesDf['Wind Speed'] < 5.5) & (citiesDf['Cloudiness'] < 25)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
101,101,Belaya Gora,68.5333,146.4167,24.43,33,5,0.07,RU,1624066866
186,186,Touros,-5.1989,-35.4608,24.50,92,16,4.34,BR,1624066794
223,223,Tutóia,-2.7619,-42.2744,25.37,85,10,5.24,BR,1624066894
224,224,Sinnamary,5.3833,-52.9500,25.56,86,15,4.43,GF,1624066894
444,444,Westport,41.1415,-73.3579,25.42,74,1,1.34,US,1624066951
496,496,Tripoli,32.8752,13.1875,24.47,59,0,4.60,LY,1624066964
501,501,Wake Forest,35.9799,-78.5097,25.86,66,1,2.06,US,1624066966


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [117]:
hotel_df['Hotel Name']=""

<ipython-input-117-16a04d88ef55>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


In [120]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {    
        "key": g_key,
}

text_url ="https://maps.googleapis.com/maps/api/place/textsearch/json"

for index,row in hotel_df.iterrows():
    try:
        location = f"({row['Latitude']}, {row['Longitude']})"
        query = f"Top hotels in {row['City']},{row['Country']}" 
        params["query"] = query
        print(params["query"])
        #params['location'] = location
        res = requests.get(text_url, params=params).json()  
        hotelname = res['results'][0]['name']
        print(hotelname)        
        #print(res)
        hotel_df.loc[index, 'Hotel Name'] = hotelname        
    except:
        print("Error! No details for ", row['City'])


Top hotels in Belaya Gora,RU
Hotel Belaya Gora
Top hotels in Touros,BR


C:\Users\srini\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Siri Paraíso Hotel
Top hotels in Tutóia,BR
Tutóia Pousada Jagatá
Top hotels in Sinnamary,GF
Hotel Du Fleuve
Top hotels in Westport,US
Delamar Southport
Top hotels in Tripoli,LY
Radisson Blu Al Mahary Hotel
Top hotels in Wake Forest,US
Fairfield Inn & Suites by Marriott Raleigh Wake Forest


In [121]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
101,101,Belaya Gora,68.5333,146.4167,24.43,33,5,0.07,RU,1624066866,Hotel Belaya Gora
186,186,Touros,-5.1989,-35.4608,24.50,92,16,4.34,BR,1624066794,Siri Paraíso Hotel
223,223,Tutóia,-2.7619,-42.2744,25.37,85,10,5.24,BR,1624066894,Tutóia Pousada Jagatá
224,224,Sinnamary,5.3833,-52.9500,25.56,86,15,4.43,GF,1624066894,Hotel Du Fleuve
444,444,Westport,41.1415,-73.3579,25.42,74,1,1.34,US,1624066951,Delamar Southport
496,496,Tripoli,32.8752,13.1875,24.47,59,0,4.60,LY,1624066964,Radisson Blu Al Mahary Hotel
501,501,Wake Forest,35.9799,-78.5097,25.86,66,1,2.06,US,1624066966,Fairfield Inn & Suites by Marriott Raleigh Wak...


In [122]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [123]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))